In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv("fake_news.csv")

In [5]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
data.shape

(20800, 5)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [11]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [13]:
data = data.drop(['id'], axis=1)

In [15]:
data = data.fillna('')

In [17]:
data['content'] = data['author']+' '+data['title']+' '+data['text']

In [19]:
data  = data.drop( ['title','author', 'text'], axis=1)

In [21]:
data.head()

,label,content
0,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,1,Consortiumnews.com Why the Truth Might Get You...
3,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,1,Howard Portnoy Iranian woman jailed for fictio...


In [23]:
data['content'] = data['content'].apply(lambda x:" ".join(x.lower() for x in x.split()))

In [25]:
data['content'] = data['content'].str.replace('[^\w\s]','')

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\bhargavi\AppData\Local\Temp\ipykernel_18348\3643324700.py:1: SyntaxWarning: invalid escape sequence '\w'
  data['content'] = data['content'].str.replace('[^\w\s]','')


In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhargavi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [31]:
from nltk.stem import WordNetLemmatizer
from textblob import Word
nltk.download('wordnet')
data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhargavi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    darrell lucus house dem aide: didn’t even see ...
1    daniel j. flynn flynn: hillary clinton, big wo...
2    consortiumnews.com truth might get fired truth...
3    jessica purkiss 15 civilian killed single u ai...
4    howard portnoy iranian woman jailed fictional ...
Name: content, dtype: object

In [32]:
X = data[['content']]
Y = data['label']

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=100, stratify = Y)

In [37]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(14560, 1)
(6240, 1)
(14560,)
(6240,)


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer(analyzer = 'word', token_pattern=r'\w{1,}', max_features=5000)
vectorization.fit(data['content'])
X_train_ifdif = vectorization.fit_transform(X_train['content'])
X_test_ifdif = vectorization.transform(X_test['content'])

In [41]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
model = PassiveAggressiveClassifier(max_iter=50)
model.fit(X_train_ifdif, Y_train)
predictions = model.predict(X_test_ifdif)
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      3116
           1       0.96      0.97      0.96      3124

    accuracy                           0.96      6240
   macro avg       0.96      0.96      0.96      6240
weighted avg       0.96      0.96      0.96      6240



In [45]:
print(metrics.confusion_matrix(Y_test, predictions))

[[2987  129]
 [ 109 3015]]


In [ ]:
from sklearn.neural_network import MLPClassifier
md1 = MLPClassifier(hidden_layer_sizes=(256,64,16),activation ='relu', solver = 'adam')
md1.fit(X_train_ifdif, Y_train)
predictions = md1.predict(X_test_ifdif)
print(metrics.classification_report(Y_test, predictions))

In [ ]:
print(metrics.confusion_matrix(Y_test,predictions))

In [ ]:
import pickle
pickle.dump(mlpclf, open("fakenews1.pkl", "wb"))